In [7]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import XMLParser
import xmltodict
import requests
import io
import matplotlib as plt
from collections import MutableMapping 

from utils.parse import get_404
from utils.parse import convert_flatten
from utils.parse import get_url_list


In [ ]:
df  = pd.read_csv('CurrentInventoryReport.csv')

In [8]:
#parser = XMLParser(encoding = 'utf-16')
tree = ET.parse("data.xml")
root = tree.getroot()

In [9]:
#xmltodict converts the xml structure to a json related format.  Problems arise with key naming convention, which will be adressed durther on
#TODO: flatten dict
with io.open('data.xml', 'r', encoding='utf-8-sig') as f:
    contents = f.read()
    xml_dict = xmltodict.parse(contents)
    f.close()

In [4]:
df2 = pd.DataFrame(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][0])
df2

,OMBControlNumber,ICRReferenceNumber,AgencyCode,Title,Abstract,ICRTypeCode,Expiration,ICRStatus,AgencyContact,PIIFlag,PrivacyActStatementFlag,AnnualFederalCostAmount,StimulusIndicator,HealthcareIndicator,DoddFrankActIndicator,AuthorizingStatutes,Burden,InformationCollections,OIRAConclusion
ExpirationDate,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,2020-06-30-04:00,Active,NaN,Uncollected,Uncollected,27900,No,No,No,NaN,NaN,NaN,NaN
Person,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,NaN,Active,"{'FirstName': 'Shethir', 'LastName': 'Riva', '...",Uncollected,Uncollected,27900,No,No,No,NaN,NaN,NaN,NaN
AuthorizingStatute,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,NaN,Active,NaN,Uncollected,Uncollected,27900,No,No,No,"{'USCode': {'USCTitle': '7', 'USCSection': '50...",NaN,NaN,NaN
BurdenResponse,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,NaN,Active,NaN,Uncollected,Uncollected,27900,No,No,No,NaN,"{'TotalQuantity': '120', 'PreviousTotalQuantit...",NaN,NaN
BurdenHour,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,NaN,Active,NaN,Uncollected,Uncollected,27900,No,No,No,NaN,"{'TotalQuantity': '104', 'PreviousTotalQuantit...",NaN,NaN
BurdenCost,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,NaN,Active,NaN,Uncollected,Uncollected,27900,No,No,No,NaN,"{'TotalAmount': '0', 'PreviousTotalAmount': '0'}",NaN,NaN
InformationCollection,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,NaN,Active,NaN,Uncollected,Uncollected,27900,No,No,No,NaN,NaN,"[{'Title': 'Tobacco Report', 'CFRCitations': {...",NaN
ConcludedDate,0581-0004,201704-0581-001,0581,Tobacco Report,The Tobacco Statistic Act of 1929 (7 U.S.C. 50...,Extension without change of a currently approv...,NaN,Active,NaN,Uncollected,Uncollected,27900,No,No,No,NaN,NaN,NaN,"{'Date': '2017-06-16-04:00', 'Time': '03:40:21..."


In [11]:
# this code fetches the server status for each url.  Be careful though because this will take about 5 hours to run.  
#TODO: multiprocess with dask
url_list = get_url_list(root)
df_url = pd.DataFrame(url_list,columns=['url'])
df_url.drop_duplicates(inplace=True)
df['404'] = df['url'].apply(lambda x: get_404(x))

In [6]:
rows = []
# Example of dictionary item with nested keys of the same value
#TODO: explore feature branch of xmltodict for potential fix
xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][0]['InformationCollections']['InformationCollection'][0]['CFRCitations']


OrderedDict([('CFRCitation',
              OrderedDict([('CFRCitation',
                            OrderedDict([('CFRTitle', '7'),
                                         ('CFRPart', '30.60')]))]))])

In [ ]:
xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][21]['InformationCollections']['InformationCollection'][1]

In [32]:
#example of a problem converting nested dictionary with same keys, the xmltodict library stores them as a list and make flattening a problem
#TODO: recursivley rename keys based on parents
convert_flatten(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][21]['InformationCollections']['InformationCollection'][1])

{'Title': 'Background/Acceptance Statement (Producer and Handler Member/Alternate)',
 'CFRCitations_CFRCitation': [OrderedDict([('CFRCitation',
                OrderedDict([('CFRTitle', '7'),
                             ('CFRPart', '930.58(c), 930.83, 930.70(b)')]))]),
  OrderedDict([('CFRCitation',
                OrderedDict([('CFRTitle', '7'),
                             ('CFRPart', '930.50(a), 930.55(b)')]))]),
  OrderedDict([('CFRCitation',
                OrderedDict([('CFRTitle', '7'),
                             ('CFRPart', '930.26, 930.23, 930.70')]))]),
  OrderedDict([('CFRCitation',
                OrderedDict([('CFRTitle', '7'),
                             ('CFRPart', '930.59(b), 930.55(c)')]))])],
 'StandardFormIndicator': 'No',
 'ObligationCode': 'Mandatory',
 'FEABusinessReferenceModule_LineOfBusiness_@Code': '105',
 'FEABusinessReferenceModule_LineOfBusiness_#text': 'Economic Development',
 'FEABusinessReferenceModule_Subfunction_@Code': '011',
 'FEABusinessReferenc

In [16]:
len(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][0]['InformationCollections']['InformationCollection'])

2

In [17]:
type(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][1]['InformationCollections']['InformationCollection'])

collections.OrderedDict

In [29]:
#iterates over each ICR element and extracts relevant data.... work in progress
#unfortunately the indexing convetion used in necessary based off of the output from xmltodict
new_list_2 = []
for t in range(len(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'])):
    try:
        if type(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t]['InformationCollections']['InformationCollection']) == list:

            for x in range(len(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t]['InformationCollections']['InformationCollection'])):
                temp_dict_2 = {}

                temp_dict_2['OMBControlNumber'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('OMBControlNumber')
                temp_dict_2['ICRReferenceNumber'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('ICRReferenceNumber')
                temp_dict_2['AgencyCode'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('AgencyCode')
                temp_dict_2['Title'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('Title')
                temp_dict_2['AnnualFederalCostAmount'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('AnnualFederalCostAmount')
                temp_dict_2.update(convert_flatten(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t]['InformationCollections']['InformationCollection'][x]))
                new_list_2.append(temp_dict_2)
                #temp_dict_2.clear()
        else:
            temp_dict_2 = {}

            temp_dict_2['OMBControlNumber'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('OMBControlNumber')
            temp_dict_2['ICRReferenceNumber'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('ICRReferenceNumber')
            temp_dict_2['AgencyCode'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('AgencyCode')
            temp_dict_2['Title'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('Title')
            temp_dict_2['AnnualFederalCostAmount'] = xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t].get('AnnualFederalCostAmount')

            temp_dict_2.update(convert_flatten(xml_dict['InformationCollectionRequestList']['InformationCollectionRequest'][t]['InformationCollections']['InformationCollection']))

            new_list_2.append(temp_dict_2)
            temp_dict_2.clear()
    except KeyError as e:
        pass
    

In [17]:
#example of a flat file. Each record refers to an "informationcollection" element in the xml source file
new_df = pd.DataFrame(new_list_2)
new_df.dropna(subset=['OMBControlNumber','NumberResponses_AnnualQuantity','BurdenHour_TotalQuantity'],inplace=True)
new_df[new_df['OMBControlNumber']=='0581-0093']

,OMBControlNumber,ICRReferenceNumber,AgencyCode,Title,AnnualFederalCostAmount,CFRCitations_CFRCitation_CFRCitation_CFRTitle,CFRCitations_CFRCitation_CFRCitation_CFRPart,StandardFormIndicator,ObligationCode,FEABusinessReferenceModule_LineOfBusiness_@Code,...,Instruments_Instrument_ElectronicCapability,Instruments_Instrument_InstrumentDocument_documentType,AffectedPublicCode_PublicCode,NumberResponses_AnnualQuantity,BurdenHour_TotalQuantity,BurdenHour_BurdenHourPerResponse_ReportingFrequencies_ReportingFrequency,BurdenCost_TotalAmount,CFRCitations_CFRCitation,Instruments_Instrument,BurdenHour_BurdenHourPerResponse_ReportingFrequencies
12,0581-0093,201808-0581-002,0581,"National Research, Promotion, and Consumer Inf...",0,NaN,NaN,No,Mandatory,105,...,NaN,NaN,Private Sector,166932,138567,"[Monthly, On occasion, Quarterly, Weekly, Annu...",0,"[{'CFRCitation': {'CFRTitle': '7', 'CFRPart': ...","[{'FormNumber': 'WAT-FHR', 'FormName': 'Waterm...",NaN
13,0581-0093,201808-0581-002,0581,"National Research, Promotion & Consumer Inform...",0,NaN,NaN,No,Voluntary,105,...,NaN,NaN,Private Sector,273586,12359,"[Monthly, Quarterly, Weekly, Annually]",0,"[{'CFRCitation': {'CFRTitle': '7', 'CFRPart': ...","[{'FormNumber': 'CN-100', 'FormName': 'Cotton ...",NaN


In [26]:
new_df.to_csv('example_flat_file.csv')

In [17]:
new_df.columns

Index(['OMBControlNumber', 'ICRReferenceNumber', 'AgencyCode', 'Title',
       'AnnualFederalCostAmount',
       'CFRCitations_CFRCitation_CFRCitation_CFRTitle',
       'CFRCitations_CFRCitation_CFRCitation_CFRPart', 'StandardFormIndicator',
       'ObligationCode', 'FEABusinessReferenceModule_LineOfBusiness_@Code',
       'FEABusinessReferenceModule_LineOfBusiness_#text',
       'FEABusinessReferenceModule_Subfunction_@Code',
       'FEABusinessReferenceModule_Subfunction_#text',
       'Instruments_Instrument_FormNumber', 'Instruments_Instrument_FormName',
       'Instruments_Instrument_URL',
       'Instruments_Instrument_AvailableElectronically',
       'Instruments_Instrument_CanSubmittedElectronically',
       'Instruments_Instrument_ElectronicCapability',
       'Instruments_Instrument_InstrumentDocument_documentType',
       'AffectedPublicCode_PublicCode', 'NumberResponses_AnnualQuantity',
       'BurdenHour_TotalQuantity',
       'BurdenHour_BurdenHourPerResponse_ReportingFre

In [19]:
new_df[['AgencyCode','BurdenHour_TotalQuantity','AnnualFederalCostAmount','NumberResponses_AnnualQuantity','BurdenCost_TotalAmount']] = new_df[['AgencyCode','BurdenHour_TotalQuantity','AnnualFederalCostAmount','NumberResponses_AnnualQuantity','BurdenCost_TotalAmount']].apply(pd.to_numeric)

In [20]:
new_df.dtypes

OMBControlNumber                                                             object
ICRReferenceNumber                                                           object
AgencyCode                                                                    int64
Title                                                                        object
AnnualFederalCostAmount                                                       int64
CFRCitations_CFRCitation_CFRCitation_CFRTitle                                object
CFRCitations_CFRCitation_CFRCitation_CFRPart                                 object
StandardFormIndicator                                                        object
ObligationCode                                                               object
FEABusinessReferenceModule_LineOfBusiness_@Code                              object
FEABusinessReferenceModule_LineOfBusiness_#text                              object
FEABusinessReferenceModule_Subfunction_@Code                                

In [21]:
gp = pd.DataFrame(new_df.groupby('AgencyCode',sort=False)['AnnualFederalCostAmount'].mean())
gp.sort_values(by=['AnnualFederalCostAmount'], ascending=False )

,AnnualFederalCostAmount
AgencyCode,
607,2.459029e+09
1651,2.318412e+08
1615,9.172434e+07
960,4.186988e+07
1220,2.668200e+07
...,...
1870,0.000000e+00
1110,0.000000e+00
1210,0.000000e+00


In [ ]:
#Below cells are quick analysis based on data available in flat file

In [37]:
gp1 = pd.DataFrame(new_df.groupby('AgencyCode',sort=False)['NumberResponses_AnnualQuantity'].mean())
gp1.sort_values(by=['NumberResponses_AnnualQuantity'], ascending=False )

,NumberResponses_AnnualQuantity
AgencyCode,
3084,8.711117e+07
3235,5.483506e+07
945,2.553700e+07
1250,2.090750e+07
1210,1.311798e+07
...,...
2133,4.625000e+01
1860,1.500000e+01
1870,1.250000e+01


In [39]:
gp1 = pd.DataFrame(new_df.groupby('AgencyCode',sort=False)['BurdenCost_TotalAmount'].mean())
gp1.sort_values(by=['BurdenCost_TotalAmount'], ascending=False )

,BurdenCost_TotalAmount
AgencyCode,
563,1.261999e+08
2126,2.415511e+07
1615,1.544195e+07
3235,1.476473e+07
2040,1.380519e+07
...,...
935,0.000000e+00
985,0.000000e+00
1905,0.000000e+00


In [ ]:
gp1 = pd.DataFrame(new_df.groupby('AgencyCode',sort=False)['BurdenHour_TotalQuantity'].mean())
gp1.sort_values(by=['BurdenHour_TotalQuantity'], ascending=False )